In [1]:
import requests
import json
import pandas as pd
import hashlib # hash 함수용 sha256 사용할 라이브러리
import random

## 전체 블록 정보 확인

In [2]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
json.loads(res.content)

{'chain': [{'index': 1,
   'nonce': 100,
   'previous_hash': 1,
   'timestamp': 1709438612.932814,
   'transactions': []}],
 'length': 1}

## transaction 입력하기

In [3]:
# HTTP 헤더 설정
headers = {'Content-Type' : 'application/json; charset=utf-8'}

# 데이터 정의
data = {
        "sender": "test_from",
        "recipient": "test_to",
        "amount": 3,
        "smart_contract": {"contract_address":"myaddress"}
}

# POST 요청 보내기
requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content

b'{"message":"Transaction will be added to Block {2}"}\n'

## 노드의 블록정보 확인 - 2

In [4]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
json.loads(res.content)

{'chain': [{'index': 1,
   'nonce': 100,
   'previous_hash': 1,
   'timestamp': 1709438612.932814,
   'transactions': []}],
 'length': 1}

##  채굴하기 - 1

In [11]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res) # 여기서 200은 HTTP 상태 코드로, 요청이 성공적으로 처리되었음을 의미

<Response [200]>


## 노드의 블록정보 확인 - 3

In [6]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
json.loads(res.content)

{'chain': [{'index': 1,
   'nonce': 100,
   'previous_hash': 1,
   'timestamp': 1709438612.932814,
   'transactions': []},
  {'index': 2,
   'nonce': 272175,
   'previous_hash': 'c19cec322addc440ebe4e88d0524eb07c08e7a4f9cb00e46e239bb36a0c616cb',
   'timestamp': 1709438652.029276,
   'transactions': [{'amount': 3,
     'recipient': 'test_to',
     'sender': 'test_from',
     'timestamp': 1709438647.8169525},
    {'amount': 0.1,
     'recipient': 'node_5000',
     'sender': 'master',
     'timestamp': 1709438652.029276}]}],
 'length': 2}

## transaction 입력 / 채굴 / 블록정보 확인

In [18]:
## transaction 입력하기 -2
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
        "sender": "test_from",
        "recipient": "test_to2",
    "amount": 30,
}
requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content


## transaction3 입력하기
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
        "sender": "test_from",
        "recipient": "test_to3",
    "amount": 300,
}
requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content


##  채굴하기
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res)


## 노드의 블록정보 확인 - 4
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)

<Response [200]>


In [15]:
res.text

'{"chain":[{"index":1,"nonce":100,"previous_hash":1,"timestamp":1709438612.932814,"transactions":[]},{"index":2,"nonce":272175,"previous_hash":"c19cec322addc440ebe4e88d0524eb07c08e7a4f9cb00e46e239bb36a0c616cb","timestamp":1709438652.029276,"transactions":[{"amount":3,"recipient":"test_to","sender":"test_from","timestamp":1709438647.8169525},{"amount":0.1,"recipient":"node_5000","sender":"master","timestamp":1709438652.029276}]},{"index":3,"nonce":408417,"previous_hash":"505d98b80070242b46c83f24066e0cc9ea1b8a27a7c5f52b6b8c6e69122c853c","timestamp":1709438660.3804889,"transactions":[{"amount":30,"recipient":"test_to2","sender":"test_from","timestamp":1709438656.178697},{"amount":300,"recipient":"test_to3","sender":"test_from","timestamp":1709438658.2238572},{"amount":0.1,"recipient":"node_5000","sender":"master","timestamp":1709438660.3804889}]},{"index":4,"nonce":-728920,"previous_hash":"bf5b0c4bb18243445110fad678abc4ede1fe4bf4858259425be0a8eba01d0be1","timestamp":1709439403.883315,"tra

## Pandas 를 활용한 거래내역 조회

In [16]:
status_json = json.loads(res.text)
status_json['chain']    
tx_amount_l = []
tx_sender_l = []
tx_reciv_l  = []
tx_time_l   = []

for chain_index in range(len(status_json['chain'])):
    chain_tx = status_json['chain'][chain_index]['transactions']
    for each_tx in range(len(chain_tx)):
        tx_amount_l.append(chain_tx[each_tx]['amount'])
        tx_sender_l.append(chain_tx[each_tx]['sender'])
        tx_reciv_l.append(chain_tx[each_tx]['recipient'])
        tx_time_l.append(chain_tx[each_tx]['timestamp'])

df_tx = pd.DataFrame()
df_tx['timestamp'] = tx_time_l  
df_tx['sender'] = tx_sender_l 
df_tx['recipient'] = tx_reciv_l
df_tx['amount'] = tx_amount_l   
df_tx

,timestamp,sender,recipient,amount
0,1.709439e+09,test_from,test_to,3.0
1,1.709439e+09,master,node_5000,0.1
2,1.709439e+09,test_from,test_to2,30.0
3,1.709439e+09,test_from,test_to3,300.0
4,1.709439e+09,master,node_5000,0.1
5,1.709439e+09,test_to,test_to2,3.0
6,1.709439e+09,master,node_5000,0.1
7,1.709439e+09,test_from,test_to2,30.0
8,1.709439e+09,test_from,test_to3,300.0
9,1.709439e+09,master,node_5000,0.1


## 거래내역 기반 계정별 잔액 조회

In [17]:
df_sended = pd.DataFrame(df_tx.groupby('sender')['amount'].sum()).reset_index()
df_sended.columns = ['user','sended_amount']
df_received= pd.DataFrame(df_tx.groupby('recipient')['amount'].sum()).reset_index()
df_received.columns = ['user','received_amount']
df_received

df_status = pd.merge(df_received,df_sended, on ='user', how=  'outer').fillna(0)
df_status['balance'] = df_status['received_amount']  - df_status['sended_amount']  
df_status

,user,received_amount,sended_amount,balance
0,node_5000,0.4,0.0,0.4
1,test_to,3.0,3.0,0.0
2,test_to2,63.0,0.0,63.0
3,test_to3,600.0,0.0,600.0
4,master,0.0,0.4,-0.4
5,test_from,0.0,663.0,-663.0
